In [2]:
# adding provider and the rest of staff necessary for working with timelines
from utils.generators import random_vm_name
from utils.providers import get_crud
from cfme.infrastructure.virtual_machines import Vm

prov = get_crud('vsphere6-nested')
if not prov.exists:
    prov.create()

vm = Vm(name=random_vm_name("tl", max_length=16), provider=prov)
print("vm name: {}".format(vm.name))
if vm.exists:
    vm.delete_from_provider()
vm.create_on_provider()

vm name: test-tl-x0dr


In [3]:
# generating some power activity management events
prov.mgmt.wait_vm_running(vm.name)
prov.mgmt.stop_vm(vm.name)
prov.mgmt.wait_vm_stopped(vm.name)
prov.mgmt.start_vm(vm.name)
prov.mgmt.wait_vm_running(vm.name)

In [4]:
# timelines filter setup and obtaining all displayed power activity events
from utils.appliance.implementations.ui import navigate_to
tl_view  = navigate_to(vm, 'Timelines')
tl_view.filter.event_type.select_by_visible_text('Management Events')
tl_view.filter.event_category.fill('Power Activity')
tl_view.filter.time_position.select_by_visible_text('centered')
tl_view.filter.apply.click()

all_vm_events = tl_view.chart.get_events('Power Activity')
# checking which events we have there
for evt in all_vm_events:
    print(evt.source_vm, evt.event_type, evt.event_source)

(u'test-tl-x0dr', u'VmPoweredOnEvent', u'VC')
(u'test-tl-x0dr', u'VmPoweredOffEvent', u'VC')
(u'test-tl-x0dr', u'VmPoweredOnEvent', u'VC')


In [6]:
# obtaining the same vm events from provider entity
tl_view  = navigate_to(prov, 'Timelines')
# Management Events and Power Activity are already chosen by default
tl_view.filter.time_position.select_by_visible_text('centered')
tl_view.filter.apply.click()

prov_events = tl_view.chart.get_events()
vm_related_events = [evt for evt in prov_events if hasattr(evt, 'source_vm') and evt.source_vm==vm.name]
for evt in vm_related_events:
    print(evt.source_vm, evt.event_type, evt.event_source)



(u'test-tl-x0dr', u'VmPoweredOnEvent', u'VC')
(u'test-tl-x0dr', u'VmPoweredOffEvent', u'VC')
(u'test-tl-x0dr', u'VmPoweredOnEvent', u'VC')


In [ ]:
#to be continued
# todo: policy events